In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
from functions import RNN_generator,createGraphRNN,init_state_update,RNN_forecast
import cPickle

In [2]:
discreteList = ['dayOfWeek','payDay','month','earthquake','type','locale','locale_name','transferred','onpromotion']

In [3]:
batch_size = 100
seq_len = 16 # same as test set for convenience
learningRate = 1e-4
epoch = 30
keep_prob = 0.75
n_layers = 2
grad_clip = 5.0
bucketSize = 10000

In [4]:
cardinalitys_X = [55, 4001, 34, 337, 2, 23, 17, 6, 18]
cardinalitys_T = [7, 2, 13, 2, 7, 4, 25, 2, 3]
dimentions_X = [2, 20, 1, 2, 1, 1, 1, 1, 1]
dimentions_T = [1, 1, 1, 1, 1, 1, 1, 1, 1]

In [5]:
dX = sum(dimentions_X)
dT = sum(dimentions_T)
d = dX + dT + 2 # 2 for two cont variables

*** Build Computation Graph ***

In [6]:
inputs,train_op,cost,saver,yhat,state = createGraphRNN(batch_size,seq_len,cardinalitys_X,cardinalitys_T,\
                                                    dimentions_X,dimentions_T,\
                                                    dX,d,keep_prob,n_layers,grad_clip)

** Training **

In [7]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [8]:
monitor = 10000

In [9]:
prefix = 'train_final'
y_np = np.loadtxt(prefix+'_Y',dtype=np.float32, delimiter=",") 
weight_np = np.loadtxt(prefix+'_Weight',dtype=np.float32, delimiter=",") 
Con_np = np.loadtxt(prefix+'_Con',dtype=np.float32, delimiter=",") 
X_np = np.loadtxt(prefix+'_X',dtype=np.int32,delimiter=",") 
Count_np = np.loadtxt(prefix+'_Count',dtype=np.int32,delimiter=",") 
Dis_np = [np.loadtxt(prefix+'_Dis'+str(j),dtype=np.int32,delimiter=",")  for j in range(len(discreteList))]

In [10]:
init_state = tuple([np.zeros((batch_size,d),dtype=np.float32) for i in range(n_layers)])
for i in range(epoch):
    cost_train = .0
    weight_train = .0
    #cost_val = .0
    #weight_val = .0
    for j,X_nps in enumerate(RNN_generator(y_np, weight_np,Con_np,Dis_np,X_np,Count_np,\
                              batch_size,seq_len,bucketSize,downSample=0.9)):
        _,cost_np,init_state = sess.run([train_op,cost,state],\
                             dict(zip(inputs,X_nps+[learningRate,init_state])))
        cost_train += cost_np
        weight_train = weight_train + np.sum(X_nps[1])
        
        if j%monitor == 0:
        
            print "Train loss:{}".format(np.sqrt(seq_len*batch_size*cost_train/weight_train))
            cost_train = .0
            weight_train = .0

Train loss:0.636511577953
Train loss:0.968842682815
Train loss:1.04433992075
Train loss:0.82437184032
Train loss:0.900429566988
Train loss:1.08760504307
Train loss:0.814018178747
Train loss:0.645312193267
Train loss:0.813628841162
Train loss:0.675931445994
Train loss:0.541603960145
Train loss:0.828752815207
Train loss:0.596420544171
Train loss:0.833746130898
Train loss:0.95236270647
Train loss:0.569947677707
Train loss:0.624942583593
Train loss:0.666621331775
Train loss:0.964010395789
Train loss:0.792642703402
Train loss:0.633126946624
Train loss:0.698547485528
Train loss:0.638245940162
Train loss:0.959674824896
Train loss:0.622096158467
Train loss:0.653523635809
Train loss:0.606417494125
Train loss:0.530936422685
Train loss:0.941848247789
Train loss:0.687379700177


In [11]:
# Save model parameters
saver.save(sess,'RNN_fillin_01')

'RNN_fillin_01'

*** Prediction ***

In [12]:
inputs,train_op,cost,saver,yhat,state = createGraphRNN(None,1,cardinalitys_X,cardinalitys_T,\
                                                    dimentions_X,dimentions_T,\
                                                    dX,d,keep_prob,n_layers,grad_clip)

In [13]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver.restore(sess,'RNN_fillin_01')

INFO:tensorflow:Restoring parameters from RNN_fillin_01


In [14]:
index = np.loadtxt('Index_final',dtype=np.int32,delimiter=",") 

In [15]:
init_tot_list = init_state_update(sess,inputs,state,batch_size*10,d,n_layers,\
                                  y_np[index],Con_np[index],X_np[index],Count_np[index],\
                                  [dis[index] for dis in Dis_np])

In [16]:
prefix = 'test_SI_final'
y_np_SI = np.loadtxt(prefix+'_Y',dtype=np.int32, delimiter=",") 
Con_np_SI = np.loadtxt(prefix+'_Con',dtype=np.float32, delimiter=",") 
X_np_SI = np.loadtxt(prefix+'_X',dtype=np.int32,delimiter=",") 
Dis_np_SI = [np.loadtxt(prefix+'_Dis'+str(j),dtype=np.int32,delimiter=",")  for j in range(len(discreteList))]

In [19]:
repeat = 10
y_SI = np.zeros_like(y_np_SI)
for i in range(repeat):
    y_SI = y_SI + RNN_forecast(sess,inputs,state,yhat,batch_size*10,n_layers,\
                             np.expand_dims(y_np[index,Count_np[index]-1],-1)\
                            ,Con_np_SI,X_np_SI,Dis_np_SI,init_tot_list)
y_SI = y_SI/repeat

In [20]:
pd.DataFrame({'id':y_np_SI.flatten(),'unit_sales':y_SI.flatten()}).to_csv('RNN_01.csv',index=False) 